## Funciones comunes

In [1]:
import pandas as pd

def cargar_excel_varias_hojas(ruta):
    hojas = pd.read_excel(ruta, sheet_name=None)
    return hojas["Posts"], hojas["Metadata"], hojas["Comentarios"]

def eliminar_columna_si_existe(df, columna):
    return df.drop(columns=[columna]) if columna in df.columns else df

def eliminar_duplicados_por_columna(df, columna, etiqueta="dataset"):
    conteo = df[columna].value_counts()
    duplicados = conteo[conteo > 1]
    if not duplicados.empty:
        print(f"\n⚠️ Duplicados detectados en {etiqueta}:")
        print(duplicados)
        df = df.drop_duplicates(subset=columna, keep="first")
        print("✅ Duplicados eliminados.")
    return df

def verificar_coincidencias(mensajes_manual, mensajes_general, etiqueta="Mensajes"):
    coincidencias = mensajes_manual & mensajes_general
    no_encontrados = mensajes_manual - mensajes_general
    print(f"\n✅ {etiqueta} etiquetados manualmente: {len(mensajes_manual)}")
    print(f"✅ {etiqueta} encontrados en Posts: {len(coincidencias)}")
    print(f"❌ {etiqueta} NO encontrados: {len(no_encontrados)}")
    return coincidencias, no_encontrados

def guardar_excel_con_hojas(ruta, df_metadata, df_posts, df_comentarios):
    with pd.ExcelWriter(ruta, engine="openpyxl") as writer:
        df_metadata.to_excel(writer, sheet_name="Metadata", index=False)
        df_posts.to_excel(writer, sheet_name="Posts", index=False)
        df_comentarios.to_excel(writer, sheet_name="Comentarios", index=False)

def verificar_duplicados_por_merge(df, columna_id, filas_antes, nombre_archivo):
    filas_despues = len(df)
    if filas_despues > filas_antes:
        print(f"⚠️ Se han añadido {filas_despues - filas_antes} filas tras el merge. Posible duplicación.")
        df_dup = df[df.duplicated(subset=[columna_id], keep=False)]
        if not df_dup.empty:
            df_dup.to_excel(nombre_archivo, index=False)
            print(f"📝 Posibles duplicados exportados a {nombre_archivo}")


## Tema

In [ ]:
ruta_base = "clasificador_analisis/clasificador/datasets_originales/"
ruta_excel_general = ruta_base + "politicos_final.xlsx"
ruta_excel_tematica = ruta_base + "temas_dataset.xlsx"
ruta_salida = ruta_base + "politicos_tema_actualizado.xlsx"

df_posts, df_metadata, df_comentarios = cargar_excel_varias_hojas(ruta_excel_general)
df_posts_original = df_posts.copy()

df_tematica = pd.read_excel(ruta_excel_tematica)
df_tematica = df_tematica.rename(columns={"Mensaje": "Contenido_Traducido", "Etiqueta": "Tema"})
df_tematica = eliminar_duplicados_por_columna(df_tematica, "Contenido_Traducido", etiqueta="df_tematica")
df_posts = eliminar_columna_si_existe(df_posts, "Tema")

df_posts = df_posts.merge(
    df_tematica[["Contenido_Traducido", "Tema"]],
    on="Contenido_Traducido",
    how="left"
)

mensajes_manual = set(df_tematica["Contenido_Traducido"].dropna())
mensajes_general = set(df_posts["Contenido_Traducido"].dropna())
coinciden, no_encontrados = verificar_coincidencias(mensajes_manual, mensajes_general, etiqueta="Mensajes")

if no_encontrados:
    df_no = df_tematica[df_tematica["Contenido_Traducido"].isin(no_encontrados)]
    df_no.to_excel(ruta_base + "mensajes_no_emparejados.xlsx", index=False)

verificar_duplicados_por_merge(
    df_posts,
    columna_id="Enlace_Post",
    filas_antes=len(df_posts_original),
    nombre_archivo=ruta_base + "posts_duplicados_por_merge.xlsx"
)

enlaces_original = set(df_posts_original["Enlace_Post"].dropna())
enlaces_actual = set(df_posts["Enlace_Post"].dropna())
enlaces_agregados = enlaces_actual - enlaces_original

print(f"\n🔍 Enlaces nuevos (no estaban en el original): {len(enlaces_agregados)}")

if enlaces_agregados:
    df_agregados = df_posts[df_posts["Enlace_Post"].isin(enlaces_agregados)]
    print("\n🔗 Enlaces agregados inesperadamente:")
    print(df_agregados[["Enlace_Post", "Contenido_Traducido", "Tema"]].to_string(index=False))
    df_agregados.to_excel(ruta_base + "posts_agregados_por_merge.xlsx", index=False)

guardar_excel_con_hojas(ruta_salida, df_metadata, df_posts, df_comentarios)



⚠️ Duplicados detectados en df_tematica:
Invasión \n Invasión \n Invasión\n Invasión\n Invasión\n Invasión\n Invasión\n Invasión\n Invasión\n Invasión    2
Name: Contenido_Traducido, dtype: int64
✅ Duplicados eliminados.

✅ Mensajes etiquetados manualmente: 918
✅ Mensajes encontrados en Posts: 918
❌ Mensajes NO encontrados: 0

🔍 Enlaces nuevos (no estaban en el original): 0


## Tono

In [ ]:
ruta_base = "clasificador_analisis/clasificador/datasets_originales/"
ruta_excel_general = ruta_base + "politicos_tema_actualizado.xlsx"
ruta_excel_tono = ruta_base + "tonos_dataset.xlsx"
ruta_salida = ruta_base + "politicos_manual_completo.xlsx"

df_posts, df_metadata, df_comentarios = cargar_excel_varias_hojas(ruta_excel_general)
df_posts_original = df_posts.copy()
df_comentarios_original = df_comentarios.copy()

df_tono = pd.read_excel(ruta_excel_tono)
df_tono = df_tono.rename(columns={"Mensaje": "Contenido_Traducido", "Etiqueta": "Tono"})

df_tono_posts = df_tono[df_tono["Tipo"] == "Post"]
df_tono_comentarios = df_tono[df_tono["Tipo"] == "Comentario"]

df_tono_posts = eliminar_duplicados_por_columna(df_tono_posts, "Contenido_Traducido", etiqueta="df_tono_posts")
df_tono_comentarios = eliminar_duplicados_por_columna(df_tono_comentarios, "Contenido_Traducido", etiqueta="df_tono_comentarios")

df_posts = eliminar_columna_si_existe(df_posts, "Tono")
df_comentarios = eliminar_columna_si_existe(df_comentarios, "Tono")

df_posts = df_posts.merge(
    df_tono_posts[["Contenido_Traducido", "Tono"]],
    on="Contenido_Traducido",
    how="left"
)

df_comentarios = df_comentarios.merge(
    df_tono_comentarios[["Contenido_Traducido", "Tono"]],
    left_on="Comentario_Traducido",
    right_on="Contenido_Traducido",
    how="left"
).drop(columns=["Contenido_Traducido"])

mensajes_posts = set(df_tono_posts["Contenido_Traducido"].dropna())
mensajes_general_posts = set(df_posts["Contenido_Traducido"].dropna())
coincidencias_posts, no_encontrados_posts = verificar_coincidencias(
    mensajes_posts, mensajes_general_posts, etiqueta="Posts"
)

mensajes_comentarios = set(df_tono_comentarios["Contenido_Traducido"].dropna())
mensajes_general_comentarios = set(df_comentarios["Comentario_Traducido"].dropna())
coincidencias_comentarios, no_encontrados_comentarios = verificar_coincidencias(
    mensajes_comentarios, mensajes_general_comentarios, etiqueta="Comentarios"
)

if no_encontrados_posts:
    df_no_post = df_tono_posts[df_tono_posts["Contenido_Traducido"].isin(no_encontrados_posts)]
    df_no_post.to_excel(ruta_base + "posts_no_emparejados_tono.xlsx", index=False)

if no_encontrados_comentarios:
    df_no_com = df_tono_comentarios[df_tono_comentarios["Contenido_Traducido"].isin(no_encontrados_comentarios)]
    df_no_com.to_excel(ruta_base + "comentarios_no_emparejados_tono.xlsx", index=False)

verificar_duplicados_por_merge(
    df_posts,
    columna_id="Enlace_Post",
    filas_antes=len(df_posts_original),
    nombre_archivo=ruta_base + "posts_duplicados_por_merge_tono.xlsx"
)

verificar_duplicados_por_merge(
    df_comentarios,
    columna_id="Enlace_Comentario",
    filas_antes=len(df_comentarios_original),
    nombre_archivo=ruta_base + "comentarios_duplicados_por_merge_tono.xlsx"
)

guardar_excel_con_hojas(ruta_salida, df_metadata, df_posts, df_comentarios)



⚠️ Duplicados detectados en df_tono_posts:
Invasión \n Invasión \n Invasión\n Invasión\n Invasión\n Invasión\n Invasión\n Invasión\n Invasión\n Invasión    2
Name: Contenido_Traducido, dtype: int64
✅ Duplicados eliminados.

⚠️ Duplicados detectados en df_tono_comentarios:
#Artinon                                                                                                                                                                                                                                                                                   7
Enhorabuena                                                                                                                                                                                                                                                                                4
DEP                                                                                                                                                         

In [4]:
path = ruta_base + "politicos_manual_completo.xlsx"

df_posts = pd.read_excel(path, sheet_name="Posts")
df_comentarios = pd.read_excel(path, sheet_name="Comentarios")

num_temas_ya_etiquetados = df_posts["Tema"].notna().sum()
num_tonos_ya_etiquetados = df_posts["Tono"].notna().sum()
comentarios_tonos_ya_etiquetados = df_comentarios["Tono"].notna().sum()

print(f"📌 Mensajes ya etiquetados con tema: {num_temas_ya_etiquetados}")
print(f"📌 Mensajes ya etiquetados con tono: {num_tonos_ya_etiquetados}")
print(f"📌 Comentarios ya etiquetados con tono: {comentarios_tonos_ya_etiquetados}")


📌 Mensajes ya etiquetados con tema: 922
📌 Mensajes ya etiquetados con tono: 922
📌 Comentarios ya etiquetados con tono: 1793
